In [1]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [2]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [3]:
# Setting a temp directory for tests
TMP=/tmp/test-trace-misc

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
alias | wc -l

0


In [4]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***
## temp_dir=$TMP/test-$$
tempRegex='^\d{2}[/-]\d{2}[/-]\d{4}$'
temp_dir=$TMP/test-$tempRegex

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-trace-misc/test-^\d{2}[/-]\d{2}[/-]\d{4}$


In [5]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [6]:
##1 SHOWS HISTORY OF BASH COMMANDS
alias hist='history $LINES'
##2 SHOWS HISTORY WITHOUT TIMESTAMPS
function h { hist | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }

## CREATES ERROR (INVALID SYNTAX)
# $ hist
# $ h

In [7]:
##3 ASCTIME
function asctime() { perl -e 'print (scalar localtime($1));'; echo ''; }
asctime | perl -pe 's/\d/N/g; s/\w+ \w+/DDD MMM/;'

DDD MMM  N NN:NN:NN NNNN


In [8]:
##4 FILTER DIRECTORY NAMES
function filter-dirnames () { perl -pe 's/\/[^ \"]+\/([^ \/\"]+)/$1/g;'; }

# awk is used for removing PID column (1st col)
ps | filter-dirnames | awk '!($1="")'

 TTY TIME CMD
 pts/3 00:00:00 bash
 pts/3 00:00:00 ps
 pts/3 00:00:00 bash
 pts/3 00:00:00 awk
 pts/3 00:00:00 perl


In [9]:
##5 COMMAIZE NUMBERS
function comma-ize-number () { perl -pe 'while (/\d\d\d\d/) { s/(\d)(\d\d\d)([^\d])/\1,\2\3/g; } '; }

echo '99012342305324254' | comma-ize-number

99,012,342,305,324,254


In [10]:
##6 APPLYING NUMERIC SUFFIXES

function apply-numeric-suffixes () {
    local just_once="$1"
    local g="g";
    if [ "$just_once" = "1" ]; then g=""; fi
    perl -pe '$suffixes="_KMGT";  s@\b(\d{4,15})(\s)@$pow = int(log($1)/log(1024));  $new_num=($1/1024**$pow);  $suffix=substr($suffixes, $pow, 1);  sprintf("%.3g%s%s", $new_num, $suffix, $2)@e'"$g;"
}

echo "8000000000" | apply-numeric-suffixes

7.45G


In [11]:
##6.1 APPLYING USAGE NUMERIC SUFFIXES
function apply-usage-numeric-suffixes() {
    perl -pe 's@^(\d+)(?=\s)@$1 * 1024@e;' | apply-numeric-suffixes 1
}

echo "8000000000" | apply-usage-numeric-suffixes
echo "8000000" | apply-usage-numeric-suffixes 

7.45T
7.63G


In [12]:
#7 usage-pp
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 8

function usage {
    output_file=$(default_assignment "$1" "usage.list")
    rename-with-file-date "$output_file";
    $NICE du --block-size=1K --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1;
    $PAGER $output_file;
}

function usage-alt {
    local output_file=$TEMP/$(basename $PWD)"-usage.list";
    usage "$output_file"
}

function byte-usage () { output_file="usage.bytes.list"; backup-file $output_file; $NICE du --bytes --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1; $PAGER $output_file; }

alias usage-pp='usage | apply-usage-numeric-suffixes | $PAGER'

In [13]:
# ERROR GENERATED - MULTIPLE
# $ ups a > psa.txt
# $ usage psa.txt
# | bash: ups: command not found
# | bash: default_assignment: command not found
# | bash: rename-with-file-date: command not found
# | bash: $output_file: ambiguous redirect
# | sort: fflush failed: 'standard output': Broken pipe
# | sort: write error

In [14]:
#8 number-columns(file): number each column in first line of tabular file
function number-columns () { head -1 "$1" | perl -0777 -pe '$c = 1; s/^/1: /; s/\t/"\t" . ++$c . ": "/eg;'; };
function number-columns-comma () { head -1 "$1" | perl -pe 's/,/\t/g;' | number-columns -; }


In [15]:
#8.1) 
rm -rf ./*
mkdir backup
printf "THIS IS THE START\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS THE END\n" > thisisatest.txt
ps -aux > process.txt

number-columns thisisatest.txt
number-columns-comma process.txt

1: THIS IS THE START
1: USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND


In [16]:
#9) reverse REVERSES cat COMMAND
alias reverse='tac'
cat thisisatest.txt
linebr
reverse thisisatest.txt

THIS IS THE START
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS THE END
--------------------------------------------------------------------------------
THIS IS THE END
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS THE START


In [17]:
#10) backup-file
function backup-file () { local file="$1"; if [ -e "$file" ]; then $BIN_DIR/dobackup.sh "$file"; fi; }

In [18]:
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 4 (OUTPUT)
backup-file thisisatest.txt

Backing up 'thisisatest.txt' to './backup/thisisatest.txt'


In [19]:
function byte-usage () { output_file="usage.bytes.list"; backup-file $output_file; $NICE du --bytes --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1; $PAGER $output_file; }

In [20]:
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 5 (OUTPUT)
byte-usage process.txt
# (Same output for 'byte-usage "Hi Mom"' and 'byte-usage thisisatest.txt')

40M	.
4.11M	./backup


In [21]:

function check-errors () { (DEBUG_LEVEL=1 check_errors.perl -context=5 "$@") 2>&1 | $PAGER; }
function check-all-errors () { (DEBUG_LEVEL=1 check_errors.perl -warnings -relaxed -context=5 "$@") 2>&1 | $PAGER; }
alias check-warnings='echo "*** Error: use check-all-errors instead ***"; echo "    check-all-errors"'
alias check-all-warnings='check-all-errors -strict'

function check-errors-excerpt () {
    local base="$TMP/check-errors-excerpt-$$"
    local head="$base.head"
    local tail="$base.tail"
    check-errors "$@" | head >| $head;
    cat "$head"
    check-errors "$@" | tail >| $tail;
    diff "$head" "$tail" >| /dev/null
    if [ $? != 0 ]; then
        echo "\$?=$?"
        cat "$tail";
    fi
}

In [22]:
#11 - LOG FILES ARE REQUIRED FOR check-errors
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 10 (OUTPUT)

check-errors process.txt
linebr
check-all-errors process.txt
linebr
check-warnings
linebr
check-all-warnings
linebr
check-errors-excerpt process.txt

process.txt
--------------------------------------------------------------------------------
process.txt
--------------------------------------------------------------------------------
*** Error: use check-all-errors instead ***
    check-all-errors
--------------------------------------------------------------------------------

usage: check_errors.perl [options]

options = [-warnings] [-context=N] [-no_astericks] [-skip_ruby_lib] [-relaxed | -strict]

ex: check_errors.perl whatever


Notes:
- The default context is 1
- Warnings are skipped by default
- Use -no_astericks if input uses ***'s outside of error contexts
Use -relaxed to exclude special cases (e.g., xyz='error')

--------------------------------------------------------------------------------
process.txt


In [23]:
function tkdiff () { wish -f $BIN_DIR/tkdiff.tcl "$@" & }
alias rdiff='rev_vdiff.sh'
alias tkdiff-='tkdiff -noopt'
function kdiff () { kdiff.sh "$@" & }
alias vdiff='kdiff'

In [24]:
# # 12) rdiff, tkdiff, vdiff - ERROR IN COMMANDS (as shown)

# $ rdiff process.txt psa.txt
# $ linebr
# $ tkdiff- process.txt psa.txt
# $ linebr
# $ vdiff process.txt 

# | Echo: Command not found.
# | ???
# | cvs: Command not found.
# | 
# | echo: No match.
# | echo: No match.
# | issuing: tkdiff.tcl -r.-1 process.txt
# | [1] 5609
# | tkdiff.tcl: Command not found.
# | --------------------------------------------------------------------------------
# | bash: tkdiff: command not found
# | --------------------------------------------------------------------------------
# | [1] 5615


In [25]:
# ERRORS UPDATED 

# $ tkdiff process.txt psa.txt
#   linebr
#   vdiff process.txt psa.txt
#   linebr
#   rdiff process.txt 
#   linebr

# | [1] 6322
# | --------------------------------------------------------------------------------
# | Error in startup script: couldn't read file "/home/xaea12/tom-project/shell-scripts/tests/../tkdiff.tcl": no such file or directory
# | [2] 6325
# | [1]   Exit 1                  wish -f $BIN_DIR/tkdiff.tcl "$@"
# | /home/xaea12/bin/kdiff.sh: line 56: /kdiff-6325.log: Permission denied
# | --------------------------------------------------------------------------------
# | [2]+  Done                    kdiff.sh "$@"
# | Echo: Command not found.
# | ???
# | cvs: Command not found.
# | 
# | echo: No match.
# | echo: No match.
# | issuing: tkdiff.tcl -r.-1 process.txt
# | [1] 6364
# | tkdiff.tcl: Command not found.
# | --------------------------------------------------------------------------------


In [26]:
# diff
diff_options="--ignore-space-change --ignore-blank-lines"
alias diff='command diff $diff_options'
alias diff-default='command diff'
alias diff-ignore-spacing='diff --ignore-all-space'

alias do-diff='do_diff.sh'

function diff-rev () {
    local diff_program="diff"
    if [ "$1" = "--diff-prog" ]; then
        diff_program="$2"
        shift 2
    fi
    local right_file="$1"
    local left_file="$2"
    
    if [ -d "$left_file" ]; then left_file="$left_file"/$(basename "$right_file"); fi
    "$diff_program" "$left_file" "$right_file"
}
alias kdiff-rev='diff-rev --diff-prog kdiff'
alias diff-log-output='compare-log-output.sh'
alias vdiff-rev=kdiff-rev

In [27]:
# # TESTING diff COMMANDS

# # EVERY DIFF COMMANDS WORK NORMALLY, BUT TESTS HALT AFTER

# linebr
ps -u > process1.txt
ps -u > process2.txt


In [28]:
# #1) diff - HALTS PROCESS (WORKS FINE)

# $ diff process1.txt process2.txt #(DIFFERENTIATING PROCESSES FOR DIFFERENT INSTANCES)

In [29]:
# #2) diff-default - HALTED (WORKS FINE)
# $ diff-default process1.txt process2.txt

In [30]:
# #3) diff-ignore-spacing -HALTED (WORKS FINE)
# $ diff-ignore-spacing process1.txt process2.txt

In [31]:
# #4) do-diff - HALTED (WORKS FINE)
# $ do-diff process1.txt process2.txt

In [32]:
# #5) kdiff-rev & vdiff-rev PERFORM THE SAME TASK - HALTS (WORKS FINE)
# $ kdiff-rev process1.txt process2.txt
# linebr
# $ vdiff-rev process1.txt process2.txt

In [33]:
# 6) diff-log-output

# $ ls
# $ linebr
# $ diff-log-output logger1.log logger2.log

# ERROR GENERATED TO BE FIXED

# bash: completion: function `_completion_loader' not found (COMPLETE COMMENT FILE RETURNS THIS)

In [34]:
mkdir -p $HOME/info
echo "THIS IS A NOICE SIGNATURE" > $HOME/info/.noice-signature

In [35]:
## TODO: echo filename and then cat??
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 6

function signature () {
    if [ "$1" = "" ]; then
        $LS "$HOME/info/.$1-signature"
        echo "Usage: signature dotfile-prefix"
        echo "ex: signature scrappycito"
        return;
    fi
    local filename="$HOME/info/.$1-signature"
    echo "$filename:"
    cat "$filename"
    }
alias cell-signature='signature cell'
alias home-signature='signature home'
alias po-signature='signature po'
alias tpo-signature='signature tpo'
alias tpo-scrappycito-signature='signature tpo-scrappycito'
alias scrappycito-signature='signature scrappycito'
alias farm-signature='signature farm'
alias circulo-signature='signature circulo'
# TODO: automatically derive aliases for ~/info/*.signature*

signature noice

/home/aveey/info/.noice-signature:
THIS IS A NOICE SIGNATURE


In [36]:
# most-recent-backup(file): returns most recent backup for FILE in ./backup, accounting for revisions (e.g., extract_matches.perl.~4~)
## BATSPP 2.1.X EXCEPTION ERROR AT LINE 13

function most-recent-backup() {
    if [ '$1' = '' ]; then
        echo 'usage: most-recent-backup filename'
        echo 'use BACKUP_DIR=dir ... to override use of ./backup'
        return
    fi
    local file='$1';
        local dir='$BACKUP_DIR'; if [ '$dir' = '' ]; then dir=./backup; fi
    
    $LS -t $dir/* | /bin/egrep '/$file(~|.~*)?' | head -1;
}

function diff-backup-helper {
    local diff='$1'; local file='$2'; 
    '$diff' $(most-recent-backup "$file") '$file';
}

alias diff-backup='diff-backup-helper diff'
alias kdiff-backup='diff-backup-helper kdiff'

$BIN_DIR/dobackup.sh process1.txt

Backing up 'process1.txt' to './backup/process1.txt'


In [37]:
ls ./backup/

process1.txt  thisisatest.txt


In [38]:
## WORKS FINE - HALTS THE NOTEBOOK TESTS
# $ diff-backup process1.txt

In [39]:
## WORKS FINE - HALTS THE NOTEBOOK TESTS
# $ kdiff-backup process1.txt